In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from pprint import pprint
import json

In [2]:
# 1 = NASA Mars News
# 2 = JPL Mars Space Images
# 3 = Mars Weather
# 4 = Mars Facts
# 5 = Mars Hemispheres

In [3]:
url1 = "https://mars.nasa.gov/news/"
url2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url3 = "https://twitter.com/marswxreport?lang=en"
url4 = "https://space-facts.com/mars/"
url5 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [4]:
def soupify(url):
    browser = Browser("chrome")
    browser.visit(url)
    html_code = browser.html
    soup = bs(html_code, "html.parser")
    return soup
    browser.windows[0].close()

soup1 = soupify(url1)
soup2 = soupify(url2)
soup3 = soupify(url3)
soup4 = soupify(url4)
soup5 = soupify(url5)

In [5]:
# 1 Mars news

In [6]:
news_title = soup1.find("div", class_="content_title").text.strip()
news_p = soup1.find("div", class_="article_teaser_body").text.strip()

In [7]:
# 2 JPL featured image

In [8]:
featured_image_url = soup2.find("article", class_="carousel_item")

In [9]:
featured_image_url = featured_image_url['style'][23:-3]

In [10]:
featured_image_url = "https://www.jpl.nasa.gov" + featured_image_url

In [11]:
# 3 Mars weather

In [12]:
mars_weather = soup3.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

In [13]:
# 4 Mars facts

In [52]:
mars_facts_dict = {}

In [15]:
mars_facts = soup4.find_all("table", class_="tablepress tablepress-id-mars")[0].find_all("tr")

In [55]:
for i in range(len(mars_facts)):

    field = mars_facts[i].find_all("td")[0].text.strip()
    value = mars_facts[i].find_all("td")[1].text.strip()
    
    print(field)
    
    field = field.replace(" ", "_")
    field = field[:-1]
    
    print(field)
    
    
    
    mars_facts_dict[field] = value
    

Equatorial Diameter:
Equatorial_Diameter
Polar Diameter:
Polar_Diameter
Mass:
Mass
Moons:
Moons
Orbit Distance:
Orbit_Distance
Orbit Period:
Orbit_Period
Surface Temperature:
Surface_Temperature
First Record:
First_Record
Recorded By:
Recorded_By


In [54]:
mars_facts_dict

{'Equatorial Diameter': '6,792 km',
 'First Record': '2nd millennium BC',
 'Mass': '6.42 x 10^23 kg (10.7% Earth)',
 'Moons': '2 (Phobos & Deimos)',
 'Orbit Distance': '227,943,824 km (1.52 AU)',
 'Orbit Period': '687 days (1.9 years)',
 'Polar Diameter': '6,752 km',
 'Recorded By': 'Egyptian astronomers',
 'Surface Temperature': '-153 to 20 °C'}

mars_facts

In [17]:
facts_df = pd.DataFrame(mars_facts_dict)

In [18]:
facts_html = facts_df.to_html()


In [19]:
# 5 Mars hemispheres

In [20]:
hemi_image_urls = []

hemi_base_url = "https://astrogeology.usgs.gov"

hemi_exts = soup5.find_all("div", class_="description")

for ext in hemi_exts:
    loop_dict = {}
    
    hemi_title = ext.a.text[:-9]
    
    hemi_ext = ext.a["href"]
    hemi_url_1 = hemi_base_url + hemi_ext
    
    hemi_soup = soupify(hemi_url_1)
    hemi_url_2 = hemi_soup.find("div", class_="downloads").a["href"]
    
    loop_dict["title"] = hemi_title
    loop_dict["img_url"] = hemi_url_2
    
    hemi_image_urls.append(loop_dict)

In [21]:
hemi_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere'}]

In [22]:
results = {}
results["news_title"] = news_title
results["news_p"] = news_p
results["featured_image_url"] = featured_image_url
results["mars_weather"] = mars_weather
results["facts_html"] = facts_html
results["hemi_image_urls"] = hemi_image_urls

In [38]:
facts_df

,Fields,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers
